Наша программа будет телеграмм-ботом с возможностью принимать фотографии от пользователя, делать на этой фотографии предсказание возраста и отправлять пользователю результат.

Для корректной работы требуется наличие в директории минимум 3 обученных нейросетей, папка *examples*, в которой будут храниться результаты предсказаний нейросети, а так же доступ к вашему гугл диску.

Нейросети можно скачать здесь https://github.com/KirillCKO/GraduateWork в папке *models.keras*

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

resnet_model = tf.keras.models.load_model('/content/drive/MyDrive/Нетология/Diploma/Новый/ResNet_age_model_crop_10.keras')
vgg_model = tf.keras.models.load_model('/content/drive/MyDrive/Нетология/Diploma/Новый/age_model_vgg_crop_9.keras')
best_age_model = tf.keras.models.load_model('/content/drive/MyDrive/Нетология/Diploma/Новый/best_age_model.keras')

Выполняем следующий код в терминале, чтобы скачать нужные пакеты библиотек:

- Устанавливаем библиотеку `imagehash` для работы с хэш-данными файлов

`!pip install imagehash`

- Устанавливаем библиотеку с обученной нейросетью `mtcnn` для детекциилиц на фото

`!pip install mtcnn`

- Устанавливаем библиотеку для работы с телеграм ботом `telebot`

`!python -m pip install pyTelegramBotAPI`

In [ ]:
# Вспомогательная функция, которая будет работать внутри основной

def process_and_save_image(image_,
                           examples_folder="/content/drive/MyDrive/Нетология/Diploma/сборка/examples"):
  """
  Обрабатывает изображение и сохраняет его, если такого изображения раньше не было

  Аргументы:
  image_ (str): Путь к изображению.
  examples_folder (str, optional): Путь к папке, в которую сохраняются примеры.
      По умолчанию "/content/drive/MyDrive/Нетология/Diploma/сборка/examples".

  Примеры использования:
      process_and_save_image("/content/image.jpg")
      process_and_save_image("/content/image.jpg", "/content/examples")
  """


  import imagehash
  from PIL import Image

  # Получаем хеш изображения
  image = Image.open(image_)
  image = image.convert("RGB")
  image_hash = imagehash.average_hash(image)

  # Получаем список файлов в папке examples
  files_in_examples = os.listdir(examples_folder)

  # Проверяем, существует ли файл с таким же хешем в папке examples
  max_file_number = 0
  for file_name in files_in_examples:
    # Ищем максимальный номер файла в папке examples
    file_number = int(file_name.split('.')[0][7:])
    max_file_number = max(max_file_number, file_number)
    # Проверяем чтобы не было копий
    file_name = examples_folder + '/' + file_name
    another = Image.open(file_name)
    another = another.convert("RGB")
    if imagehash.average_hash(another) == image_hash:
      print(f'Файл {file_name} есть в директории')
      break

  else:
    # Генерируем имя нового файла
    new_file_name = f"Example{max_file_number + 1}.jpg"

    # Сохраняем изображение в папку examples
    image.save(os.path.join(examples_folder, new_file_name))
    print(f"Изображение {new_file_name} добавлено в папку examples.")

In [ ]:
def how_many(image_path):
  """
  Функция проверяет сколько лиц было обнаружено на фото
  """

  import cv2
  import io
  import numpy as np
  from mtcnn import MTCNN

  # Преобразуем изображение в массив
  image = cv2.imread(image_path)
  detector = MTCNN()
  faces = detector.detect_faces(image)

  how_many_faces = len(faces)

  if how_many_faces > 1:
    detections = np.copy(image)

    for i, face in enumerate(faces):
      x, y, w, h = face['box']
      # отрисовываем детекцию лиц
      cv2.rectangle(detections,(x,y),(x+w,y+h),(255,0,0), 2)
      img = io.BytesIO()
      plt.imshow(detections[..., ::-1])
      plt.text(x, y, f'{i+1}',
              color='black', fontsize=12,
              fontweight='bold', ha='left',
              va='bottom', backgroundcolor='0.9')
      ax = plt.gca()
      ax.set_axis_off()
      plt.savefig(img, format='png', bbox_inches='tight', pad_inches=0)
    process_and_save_image(img)
    img.seek(0)

  else: img = False

  plt.cla()
  return img, how_many_faces

In [ ]:
# Основная функция, принимает фото - выдает результат работы нейросети
def gen_age(image_path, mode='*Standart*', choise=False):
    """
    Функция делает предсказание для одного изображения с помощью двух моделей ResNet и VGG,
    а затем печатает изображение с указанием предсказанного возраста.

    Параметры:
    image_path (str): Путь к изображению.
    """
    import cv2
    import io
    import numpy as np
    from mtcnn import MTCNN

    # Преобразуем изображение в массив
    image = cv2.imread(image_path)
    detector = MTCNN()
    faces = detector.detect_faces(image)

    if len(faces) > 0:
      detections = np.copy(image)

      for i, face in enumerate(faces):
        x, y, w, h = face['box']
        if choise:
          if i+1 != int(choise): continue
      # отрисовываем детекцию лиц
        cv2.rectangle(detections,(x,y),(x+w,y+h),(255,0,0), 2)

        x1, y1 = x, y
        x2, y2 = x+w, y+h

        img_ = image[y1:y2, x1:x2]
        img_ = cv2.resize(img_, (256, 256))
        img_ = img_ / 255.0


        # Делаем предсказания для каждой модели
        resnet_prediction = resnet_model.predict(np.expand_dims(img_, axis=0), verbose=0)
        vgg_prediction = vgg_model.predict(np.expand_dims(img_, axis=0), verbose=0)
        if mode != '*Standart*':
          predicted_age = int(resnet_prediction[0][0] + vgg_prediction[0][0])//2
        else:
          prepredict = np.array( (resnet_prediction[0][0], vgg_prediction[0][0]) )
          predicted_age = int(best_age_model.predict(np.expand_dims(prepredict, axis=0), verbose=0)[0][0])
        img = io.BytesIO()
        plt.imshow(detections[..., ::-1])
        plt.text(0.5, 1.1, f'Предсказанный возраст: {predicted_age}',
                color='black', fontsize=10,
                  ha='center', fontweight='bold',
                  va='top', transform=plt.gca().transAxes)
        ax = plt.gca()
        ax.set_axis_off()
        plt.savefig(img, format='png', bbox_inches='tight', pad_inches=0.1)

      process_and_save_image(img)
      plt.cla()
      img.seek(0)
      return img


    else: return False

Запускаем следующую ячейку чтобы бот начал свою работу.

In [ ]:
import telebot
import random
from telebot import types


token = "7102479784:AAGtBTO2adSA5Bm-e180IMfIUTqHDb-cpgM"
my_site = 'https://github.com/KirillCKO/FirstProject'

# Инициализация класса телебота
bot = telebot.TeleBot(token)

help_text = """Бот создан для определения возраста человека на фотографии.\n
Пожалуйста, пришлите ваше фото, а бот попробует предсказать возраст этого человека.\n
На данный момент точность предсказаний зависит от выбранного вами режима работы, \
а так же от качества фотографии.\n
Мы ежедневно работаем над улучшением модели, поэтому будем рады вашей обратной связи\
 она поможет нам улучшить качество предсказаний"""

start_text = f"""Привет, , выбери одну из кнопок для начала работы"""

Standart = "*Standart*"
Average = "*Average*"
global mode
global not_mode
mode = Average
not_mode = Average if mode==Standart else Standart

send_photo_text = """Для предсказания возраста по вашему фото - просто отправьте мне его 😎, других действий не требуется!

Я постоянно совершенствуюсь, поэтому мои предсказания становятся лучше 🧠"""

b_1 = types.KeyboardButton('Отправить фото')
b_2 = types.KeyboardButton('Инфо о боте')
b_3 = types.KeyboardButton('Пример работы')
b_4 = types.KeyboardButton('Исходный код')
b_5 = types.KeyboardButton('Создатель')
b_6 = types.KeyboardButton('Режим работы')


# Обработка сообщения после команд 'start', 'main', 'hello'
@bot.message_handler(commands=['start', 'main', 'hello'])
def start(message, start_text_=start_text):
    "Выводит справочное сообщение при команде /start и показывает клавиатурные кнопки"
    markup = types.ReplyKeyboardMarkup()
    # инициализация кнопок
    but_1 = types.KeyboardButton('Отправить фото')
    but_2 = types.KeyboardButton('Инфо о боте')
    but_3 = types.KeyboardButton('Пример работы')
    but_4 = types.KeyboardButton('Исходный код')
    but_5 = types.KeyboardButton('Создатель')
    but_6 = types.KeyboardButton('Режим работы')
    # Создание рядов с кнопками
    markup.row(but_1, but_2)
    markup.row(but_3, but_4)
    markup.row(but_5, but_6)
    start_text_ = start_text[:8] + message.from_user.first_name + start_text[8:]
    bot.send_message(message.chat.id,
                     start_text_,
                     reply_markup=markup)


# Обработка сообщения после команд 'site', 'code', 'git', 'github'
@bot.message_handler(commands=['site', 'code', 'git', 'github'])
def site(message):
    "Отправляем пользователя на гитхаб с иходным кодом нейросети"
    markup = types.InlineKeyboardMarkup()
    # инициализация кнопок
    but_1 = types.InlineKeyboardButton('Да', my_site)
    but_2 = types.InlineKeyboardButton('Нет', callback_data='cancel')
    # Создание рядов для этих кнопок
    markup.row(but_1)
    markup.row(but_2)
    bot.send_message(message.chat.id,
                     'Хотите перейти на сайт?',
                     reply_markup=markup)


# Декоратор для обработки любой callback_data
@bot.callback_query_handler(func=lambda callback: True)
def callback_message(callback):
    if callback.data == 'cancel':
        bot.delete_message(callback.message.chat.id, callback.message.message_id)
        bot.delete_message(callback.message.chat.id, callback.message.message_id-1)
        bot.send_message(callback.message.chat.id, "Переход на сайт отменен.")
    elif callback.data == 'cancelpredict':
        bot.delete_message(callback.message.chat.id, callback.message.message_id)
        bot.delete_message(callback.message.chat.id, callback.message.message_id-1)
        bot.send_message(callback.message.chat.id, "Обработка фото отменена.")
    elif callback.data == 'switch':
        global mode
        global not_mode
        not_mode = Average if mode==Standart else Standart
        mode = not_mode
        not_mode = Average if mode==Standart else Standart
        bot.delete_message(callback.message.chat.id, callback.message.message_id)
        bot.delete_message(callback.message.chat.id, callback.message.message_id-1)
        bot.send_message(callback.message.chat.id, f"Режим работы изменен на {mode}",
                         parse_mode='markdown')
    elif callback.data == 'notswitch':
        bot.delete_message(callback.message.chat.id, callback.message.message_id)
        bot.delete_message(callback.message.chat.id, callback.message.message_id-1)
        bot.send_message(callback.message.chat.id, "Изменение режима работы отклонено.")


# Команда 'help'
@bot.message_handler(commands=['help'])
def help(message):
    "Выводим справочное сообщение"
    bot.send_message(message.chat.id, help_text)


# Информация о чате
@bot.message_handler(commands=['chatinfo'])
def chatinfo(message):
    "Сообщение о метадате"
    bot.send_message(message.chat.id, message)


# обработка фотографий (предсказание возраста)
@bot.message_handler(content_types=['photo', 'document'])
def neuralnetwork(message, choise=False, id=False):
    "Получает входящее изображение, отправляет предсказание возраста"
    text_ = """Я не могу обработать этот файл. Почему это случилось:
    - Размер файла слишком большой
    - Файл не является фотографией

Поставьте галочку в пункте 'Сжать изображение' перед отправкой"""
    zero_people_text = "Я не вижу людей, простите"
    why_file = """Простите что-то не так с вашим файлом. Я не могу обработать его.
Попробуйте отправить ваш файл в другом формате, например jpg или png """
    global mode
    if '/content/photo.jpg' == message:
      photo_ = gen_age(message, mode=mode, choise=choise)
      bot.send_photo(id, photo_)
    else:
      try:
        try:
          file_id = message.photo[-1].file_id
        except:
          file_id = message.document.file_id
        file_info = bot.get_file(file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        if 'RGB' in str(downloaded_file) or 'JFIF' in str(downloaded_file):
          with open('photo.jpg', 'wb') as new_file:
              new_file.write(downloaded_file)
          image_path = '/content/' + 'photo.jpg'
          file_, num_people = how_many(image_path)
          if num_people == 1:
            photo_ = gen_age(image_path,  mode=mode)
            if photo_:
              bot.send_photo(message.chat.id, photo_)
            else: bot.send_message(message.chat.id, text_)
          elif num_people > 1:
            if str(num_people)[-1] in ['2','3', '4']: people = ' человека'
            else: people = ' человек'
            how_many_text = f"На фото обнаружено {num_people}{people}, кого выбираем?"
            markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
            but_0 = types.KeyboardButton('1')
            but_1 = types.KeyboardButton('2')
            but_2 = types.KeyboardButton('3')
            but_3 = types.KeyboardButton('4')
            but_4 = types.KeyboardButton('5')
            but_5 = types.KeyboardButton('6')
            but_6 = types.KeyboardButton('7')
            but_7 = types.KeyboardButton('8')
            if num_people == 2: markup.row(but_0, but_1)
            elif num_people == 3: markup.row(but_0, but_1, but_2)
            elif num_people == 4: markup.row(but_0, but_1); markup.row(but_2, but_3)
            elif num_people == 5: markup.row(but_0, but_1); markup.row(but_2, but_3); markup.row(but_4)
            elif num_people == 6: markup.row(but_0, but_1, but_2); markup.row(but_3, but_4, but_5)
            elif num_people == 7: markup.row(but_0, but_1); markup.row(but_2, but_3); markup.row(but_4, but_5); markup.row(but_6)
            elif num_people == 8: markup.row(but_0, but_1); markup.row(but_2, but_3); markup.row(but_4, but_5); markup.row(but_6, but_7)
            bot.send_photo(message.chat.id,
                          file_,
                          caption=how_many_text,
                          reply_markup=markup, )
            # Регистрируем следующую функцию, которая отработает после нажатия на кнопку
            bot.register_next_step_handler(message, on_click,  image_path, message.chat.id)
          else: bot.send_message(message.chat.id, zero_people_text)
        else: bot.send_message(message.chat.id, why_file)
      except:
        bot.send_message(message.chat.id, why_file)
def on_click(message, image_path, id): # Ф-я реагирующая на клик пользователя
  print(image_path, message.text)
  if message.text in ['0','1','2','3','4','5','6','7']:
    neuralnetwork(image_path, message.text, id)



def just_text(message):
    text_ = 'Простите, я не знаю такой команды, пожалуйста выберете одну из кнопок ниже'
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    but_1 = types.KeyboardButton('Отправить фото')
    but_2 = types.KeyboardButton('Инфо о боте')
    but_3 = types.KeyboardButton('Пример работы')
    but_4 = types.KeyboardButton('Исходный код')
    but_5 = types.KeyboardButton('Создатель')
    but_6 = types.KeyboardButton('Режим работы')
    markup.row(but_1, but_2)
    markup.row(but_3, but_4)
    markup.row(but_5, but_6)
    bot.send_message(message.chat.id, text_, reply_markup=markup)


def example(message):
    # directory = 'Examples/'
    directory = '/content/drive/MyDrive/Нетология/Diploma/сборка/examples/'
    all_images = os.listdir(directory)
    pick = directory + random.choice(all_images)
    file_ = open(pick, 'rb')
    bot.send_photo(message.chat.id, file_)

@bot.message_handler(commands=['mode'])
def switch_mode(message):
  text_ = f"""У меня есть 2 режима работы {Standart} или {Average}. Standart работает более стабильно чем Average.

Однако в некоторых случаях Average может предсказать возраст почти в 2 раза точнее. Вы можете переключаться между этими режимами.

Сейчас возраст предсказывает нейросеть {mode}, хотите переключить режим на {not_mode}?"""
  markup = types.InlineKeyboardMarkup()
  but_1 = types.InlineKeyboardButton('Да', callback_data='switch')
  but_2 = types.InlineKeyboardButton('Нет', callback_data='notswitch')
  markup.row(but_1)
  markup.row(but_2)
  bot.send_message(message.chat.id, text_, reply_markup=markup, parse_mode='markdown')


@bot.message_handler(commands=['creator'])
def creator(message):
  bot.send_message(message.chat.id, "Мой создатель - @Kyryll_ck")


@bot.message_handler(content_types=['text'])
def echo(message):
    if message.text == 'Отправить фото':
      bot.send_message(message.chat.id, send_photo_text)
    elif message.text == 'Инфо о боте': help(message)
    elif message.text == 'Пример работы': example(message)
    elif message.text == 'Исходный код': site(message)
    elif message.text == 'Создатель': creator(message)
    elif message.text == 'Режим работы': switch_mode(message)
    else: just_text(message)


bot.polling(none_stop=True)